In [1]:
cd ml-languagemodel

/home/jjjeevan/ml-languagemodel


In [ ]:
import pandas as pd
from pathlib import Path

In [ ]:
mldata = pd.read_csv('AllWiki_cleaned.csv')

In [ ]:
mldata

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_t = train_test_split(mldata, test_size=0.2, random_state=42)
X_test, X_valid = train_test_split(X_t, test_size=0.5, random_state=42)


X_tr1, X_tr2 = train_test_split(X_train, test_size=0.5, random_state=42)
X_train_split1, X_train_split2 = train_test_split(X_tr1, test_size=0.5, random_state=42)
X_train_split3, X_train_split4 = train_test_split(X_tr2, test_size=0.5, random_state=42)

In [ ]:
X_train_split1.to_csv('flair_data/train/train_split1.txt', index=False)
X_train_split2.to_csv('flair_data/train/train_split2.txt', index=False)
X_train_split3.to_csv('flair_data/train/train_split3.txt', index=False)
X_train_split4.to_csv('flair_data/train/train_split4.txt', index=False)
X_test.to_csv('flair_data/test.txt', index=False)
X_valid.to_csv('flair_data/valid.txt', index=False)

In [ ]:
type(mldata.iloc[1]['text']) == str

In [ ]:
# make an empty character dictionary
from flair.data import Dictionary
import re

char_dictionary: Dictionary = Dictionary()

    
# counter object
import collections
counter = collections.Counter()

processed = 0
tokens = 0


for i in range(len(mldata)):
    tkn = mldata.iloc[i]['text']
    if type(tkn) == str:
        tkn = re.split('\.|\s',tkn)
        processed += 1            
        chars = list(tkn)
        tokens += len(tkn)

        # Add chars to the dictionary
        counter.update(chars)

    # comment this line in to speed things up (if the corpus is too large)
    # if tokens > 50000000: break

# break

total_count = 0
for letter, count in counter.most_common():
    total_count += count

print(total_count)
print(processed)

sum = 0
idx = 0
for letter, count in counter.most_common():
    sum += count
    percentile = (sum / total_count)

    # comment this line in to use only top X percentile of chars, otherwise filter later
    # if percentile < 0.00001: break

    char_dictionary.add_item(letter)
    idx += 1
    #print('%d\t%s\t%7d\t%7d\t%f' % (idx, letter, count, sum, percentile))

print(char_dictionary.item2idx)

In [ ]:
char_dictionary.item2idx

In [ ]:
import pickle
with open('flair_chars', 'wb') as f:
    mappings = {
        'idx2item': char_dictionary.idx2item,
        'item2idx': char_dictionary.item2idx
    }
    pickle.dump(mappings, f)

In [2]:
import pickle
from flair.data import Dictionary

dictionary = Dictionary.load_from_file('flair_data/flair_chars')

I1230 14:44:07.899512 46912496440640 file_utils.py:35] PyTorch version 1.3.1 available.
/share/apps/rc/software/Anaconda3/5.3.1/envs/DeepNLP/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/share/apps/rc/software/Anaconda3/5.3.1/envs/DeepNLP/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/share/apps/rc/software/Anaconda3/5.3.1/envs/DeepNLP/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will

In [3]:
from flair.models import LanguageModel
from flair.trainers.language_model_trainer import LanguageModelTrainer, TextCorpus

# are you training a forward or backward LM?
is_forward_lm = True

In [4]:
# get your corpus, process forward and at the character level

corpus = TextCorpus('flair_data/',
                    dictionary,
                    is_forward_lm,
                    character_level=True)

2019-12-30 14:44:12,884 read text file with 6833 lines
2019-12-30 14:45:49,742 read text file with 6832 lines


In [5]:
# instantiate your language model, set hidden size and number of layers
language_model = LanguageModel(dictionary,
                               is_forward_lm,
                               hidden_size=1024,
                               nlayers=10)

In [7]:
# train your language model
trainer = LanguageModelTrainer(language_model, corpus)

trainer.train('resources/taggers/language_model',
              sequence_length=128,
              mini_batch_size=32,
              max_epochs=10)

2019-12-30 14:48:40,870 read text file with 13663 lines
2019-12-30 14:48:40,878 read text file with 13664 lines
2019-12-30 14:48:40,883 shuffled
2019-12-30 14:48:40,890 shuffled
2019-12-30 14:51:45,336 Sequence length is 128
2019-12-30 14:51:45,471 Split 1	 - (14:51:45)
2019-12-30 14:51:45,484 read text file with 13664 lines
2019-12-30 14:51:45,496 shuffled


RuntimeError: CUDA out of memory. Tried to allocate 25.75 GiB (GPU 0; 15.90 GiB total capacity; 8.34 GiB already allocated; 6.84 GiB free; 15.05 MiB cached)

In [9]:
import torch

t = torch.cuda.get_device_properties(0).total_memory/1e9
c = torch.cuda.memory_cached(0)/1e9
a = torch.cuda.memory_allocated(0)/1e9
f = c-a  # free inside cache

In [10]:
print(t, c, a, f)

17.071734784 8.973713408 8.957935616 0.015777791999999735


In [11]:
torch.cuda.device_count()

2

In [ ]:
torch.cuda.empty_cache()